In [275]:
import os, json, glob
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy.ndimage

import pyusct
from pyusct.preprocess import load_usctsim_result, draw_input

%matplotlib inline


### parameters

In [300]:
sim_result_dirs = glob.glob("/mnt/nas/nb_usctsim/sim_001/trial*")
sim_result_dirs.sort()
sim_result_dirs = sim_result_dirs[:1]
sim_result_dirs

['/mnt/nas/nb_usctsim/sim_001/trial_001']

In [301]:
out_dir = "/mnt/nas/PYUSCT_train/dataset003/"

In [302]:
# 入力RFデータ抽出範囲
offset_min = -100
offset_max = 100
offset_int = 5

# 出力画像サイズ
radius = 2

# 出力画像サンプリング数
num_sampling = 10

### 教師データ生成

In [101]:
def norm_mat(mat):
    r = mat.max() - mat.min()
    if r == 0 : pass
    return (mat - np.min(mat)) / r

In [303]:
# 出力先ディレクトリ
if not os.path.exists(out_dir): 
    os.makedirs(os.path.join(out_dir, "input"))   # 入力データ
    os.makedirs(os.path.join(out_dir, "output"))  # 出力データ  
    
# パラメータ設定保存
p = {}
p["offset"] = {"min": offset_min, "max":offset_max, "interval":offset_int}
p["outrad"] = radius
with open(os.path.join(out_dir,"param.json"), 'w') as outf:
    json.dump(p, outf, indent=4)

cnt = 0
with open(os.path.join(out_dir, "list.csv"), 'w') as outf: #
    outf.write('id,source,ix,iy,x,y\n')

    for sim_result_dir in sim_result_dirs:

        # シミュレーション結果の読み込み
        param, arr_cart_pos, mask_points, map_speed, map_dens, rfdata = load_usctsim_result(sim_result_dir)

        # インピーダンス画像
        map_impedance = np.array(map_dens, dtype=np.float64) * np.array(map_speed, dtype=np.float64)
        map_impedance = norm_mat(map_impedance)
        #     plt.imshow(map_impedance)
        #     plt.colorbar()

        # インピーダンス勾配画像
        map_grad = abs(scipy.ndimage.filters.gaussian_gradient_magnitude(map_impedance, sigma=1))
        #     plt.imshow(map_grad)
        #     plt.colorbar()

        for i in range(num_sampling): # 出力画像内サンプリング

            ix = np.random.randint(int(param["grid"]["Nx"]*0.25), int(param["grid"]["Nx"]*0.75))
            iy = np.random.randint(int(param["grid"]["Ny"]*0.25), int(param["grid"]["Ny"]*0.75))

            x = (ix - param["grid"]["Nx"]/2) * param["grid"]["dx"]
            y = (iy - param["grid"]["Ny"]/2) * param["grid"]["dy"]

            out_image = map_grad[iy-radius:iy+radius+1, ix-radius:ix+radius+1]

            _, submat = rfdata.getPointSubset([y, x], np.arange(offset_min,offset_max, offset_int))

            np.save(os.path.join(os.path.join(out_dir,"input"), '{0:0>6}'.format(i)), submat)
            np.save(os.path.join(os.path.join(out_dir,"output"),'{0:0>6}'.format(i)), out_image)

            outf.write('{0},{1},{2},{3},{4},{5}\n'.format(cnt, sim_result_dir, ix,iy,x,y))
            cnt += 1